In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [ ]:
%load_ext nb_mypy

# The Three Greedy Thieves

Three greedy thieves must cross a river, each possessing a bag of gold coins:
* Aaron has 1,000 gold coins.
* Benjamin has 700 gold coins.
* Chaim has 300 gold coins.

There is a boat available that can carry either two people or one person along with a bag of gold coins. The boat can transport two entities at a time, meaning either two thieves or one thief and a bag can cross together. The challenge arises if a thief, or a pair of thieves, is left with a quantity of gold greater than their own, they will abscond with it.

The question is whether there is a strategy that allows all three greedy thieves to cross the river without losing their respective amounts of gold.

The dictionary `Gold` stores the amount of gold coins that every thief possesses.

In [ ]:
Gold: dict[str, int] = { 'A': 1000, 'B': 700, 'C': 300 }

A state is represented as a triple of the form $(\texttt{Thieves}, \texttt{Bags}, \texttt{boat})$.  Here,
  - `Thieves` is the set of thieves on the left shore,
  - `Bags`    is the set of bags of gold coins on the left shore, and
  - `boat`    is the number of boats on the left shore.
  
The sets have to be stored as frozen sets since states have to be stored in sets in the *breadth first algorithm*.

In [ ]:
State = tuple[frozenset[str], frozenset[int], int]

The function `problem(S)` returns `True` if there is a problem on on the left shore in the state `S`.

In [ ]:
def problem(S: State) -> bool: 
    "your code here"

The function `right_shore(S)` takes a state `S` and returns the state that results from switching the left and the right side. 

In [ ]:
def right_shore(S: State) -> State:
    "your code here"

The function `no_problem(S)` is true if there is no problem on either side of the river in state `S`.

In [ ]:
def no_problem(S: State) -> bool: 
    return not problem(S) and not problem(right_shore(S))

In [ ]:
from typing import TypeVar, Iterable

In [ ]:
E = TypeVar('E')

The function `arb` takes an *iterable* `S` as its first argument and returns an arbitrary element from `S`.
If `S` is empty, `None` is returned.  In this program, `arb` will only be called with a non-empty argument `S`.

In [ ]:
def arb(S: Iterable[E]) -> E:
    for x in S:
        return x
    return None # type: ignore

The function `power` takes a frozen set `S` of elements.  It returns the *power set* of `S`, 
i.e. it returns the set of all subsets of `S`. 

In [ ]:
def power(S: frozenset[E]) -> set[frozenset[E]]:
    if len(S) == 0:
        return { frozenset() }
    else:
        x = arb(S)
        P = power(S - {x})
        return P | { M | {x} for M in P }

In [ ]:
power(frozenset({'A', 'B', 'C'}))

The function `boat_ok(Thieves, Bags)` checks two conditions:
* There has to be at least one thief on the boat.
* There can be at most two items on the boat.

In [ ]:
def boat_ok(Thieves: frozenset[str], Bags: frozenset[int]) -> bool:
    return 1 <= len(Thieves) and len(Thieves) + len(Bags) <= 2

The function `next_states` takes a state `S` and computes the set of states that can be reached from `S` by crossing the river. 

In [ ]:
def next_states(S: State) -> set[State]:
    "your code here"

Initially, all thieves, their bags, and the boat are on the left shore.
The goal is to have all thieves and their bags on the right shore, hence nothing is left on the left shore.

In [ ]:
start: State = (frozenset({'A', 'B', 'C'}), frozenset({1000, 700, 300}), 1)
goal:  State = (frozenset(), frozenset(), 0)

In [ ]:
next_states(start)

In [ ]:
next_states(goal)

# Printing the Solution

To begin with, we display the transition relation that is generated by the function `next_states`.  To this end, we need the module `graphviz`.

In [ ]:
import graphviz as gv

The function `stateToStr` takes a state and converts it into a string that can be used as a label for the node in the search graph. 

In [ ]:
def stateToStr(S: State) -> str:
    Thieves, Bags, boat = S
    Thieves = list(Thieves)
    Thieves.sort()
    Bags = list(Bags)
    Bags.sort()
    boat = ' _' if boat == 1 else ''
    result = ''
    for t in Thieves:
        result += t
    result += ' '    
    for b in Bags:
        result += str(b//100) + ','
    return result[:-1] + boat

In [ ]:
stateToStr(start)

The function `dot_graph(R)` turns a given binary relation `R` into a graph.
We define the type alias `Relation` as a set of pairs of `States`.
Unfortunately, we are not able to specify the return type, as the module `graphviz` is not yet 
equipped with type annotations.

In [ ]:
Relation = set[tuple[State, State]]

In [ ]:
frozenset({1000, 700, 300})

In [ ]:
def dot_graph(R: Relation) -> gv.Digraph:
    """This function takes a binary relation R as inputs and shows this relation as
       a graph using the module graphviz.
    """
    dot = gv.Digraph()
    dot.attr(rankdir='LR')
    Nodes = { stateToStr(a) for (a,b) in R } | { stateToStr(b) for (a,b) in R }
    for n in Nodes:
        dot.node(n)
    for (x, y) in R:
        dot.edge(stateToStr(x), stateToStr(y))
    return dot

The function call `createRelation(start)` computes the transition relation.  It assumes that all states are reachable from `start`. 

In [ ]:
def createRelation(start: State) -> Relation:
    oldM: set[State] = set()
    M:    set[State] = { start }
    while True:
        oldM = M.copy()
        M |= { y for x in M
                 for y in next_states(x)
             }
        if M == oldM:
            break
    R: Relation = set()
    for S in M:
        for SX in next_states(S):
            R.add((S, SX))
    return R

In [ ]:
createRelation(start)

In [ ]:
def fillCharsLeft(x: str, n: int) -> str:
    s = str(x)
    m = n - len(s)
    return m * " " + s

In [ ]:
def fillCharsRight(x: str, n: int) -> str:
    s = str(x)
    m = n - len(s)
    return s + m * " "

In [ ]:
def fillCharsBoth(x: str, n: int) -> str:
    s  = str(x)
    ml = (n     - len(s)) // 2
    mr = (n + 1 - len(s)) // 2
    return ml * " " + s + mr * " "

The function `printState(m, k, b)` displays a state where there are `m` missionaries, 
`k` cannibals, and `b` boats on the left shore.

In [ ]:
def printState(S: State) -> None:
    left = stateToStr(S)
    right = stateToStr(right_shore(S))
    print(fillCharsRight(left, 15) + "    |~~~~~~~|    " + fillCharsLeft(right, 15))

In [ ]:
def boatToStr(Thieves: frozenset[str], Bags: frozenset[int]) -> str:
    Thieves, Bags
    Thieves = list(Thieves)
    Thieves.sort()
    Bags = list(Bags)
    Bags.sort()
    result = ''
    for t in Thieves:
        result += t
    result += '|'    
    for b in Bags:
        result += str(b//100) + ','
    return result[:-1]

The function `printBoat(m1, k1, b1, m2, k2, b2)` prints the boat when there are
`m1` missionaries, `k1` cannibals, and `b1` boats on the left shore before the transition,
while there `m2` missionaries, `k2` cannibals, and `b2` boats on the left shore after the transition.
The function also checks whether the transition is possible at all.

In [ ]:
def printBoat(Before: State, After: State) -> None:
    ThievesBefore, BagsBefore, boatsBefore = Before
    ThievesAfter,  BagsAfter,  boatsAfter  = After
    if boatsBefore == 1:
        ThievesBoat = ThievesBefore - ThievesAfter
        BagsBoat    = BagsBefore    - BagsAfter
        print(20*" " + "> " + boatToStr(ThievesBoat, BagsBoat) + " >")
    else:
        ThievesBoat =  ThievesAfter - ThievesBefore
        BagsBoat    =  BagsAfter    - BagsBefore 
        print(20*" " + "< " + boatToStr(ThievesBoat, BagsBoat) + " <")

The function call `printPath(Path)` prints the solution of the search problem.

In [ ]:
def printPath(Path: list[State]) -> None:
    print("Solution:\n")
    for i in range(len(Path) - 1):
        Before = Path[i]
        After  = Path[i+1]
        printState(Before)
        printBoat(Before, After)
    printState(Path[-1])